In [ ]:
#para hacer la conexión con mysql
!pip install pymysql



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. CONFIGURAR CONEXIÓN A MYSQL
USER = "root"          # cámbialo si usas otro usuario
PASSWORD = "xxxx"   # pon tu contraseña real
HOST = "localhost"
PORT = "3306"
DB_NAME = "todos_a_bordo"  # la BD que creaste en Workbench

# Cadena de conexión MySQL + PyMySQL
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")

# 2. FUNCIONCITA AUXILIAR PARA LIMPIAR NOMBRES DE COLUMNAS
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
    )
    return df

# 3. CARGAR Y SUBIR TURISTAS INTERNACIONALES JULIO
df_tur_int = pd.read_excel("turistas_internacionales_julio_clean.xlsx")
df_tur_int = clean_columns(df_tur_int)

df_tur_int.to_sql(
    "turistas_internacionales_julio",  # nombre de tabla en MySQL
    engine,
    if_exists="replace",               # "replace" para sobrescribir si ya existe
    index=False
)

# 4. CARGAR Y SUBIR TURISTAS NACIONAL JULIO
df_tur_nac = pd.read_excel("turistas_nacional_julio_clean.xlsx")
df_tur_nac = clean_columns(df_tur_nac)

df_tur_nac.to_sql(
    "turistas_nacional_julio",
    engine,
    if_exists="replace",
    index=False
)

# 5. CARGAR Y SUBIR CLICK & BOAT LIMPIO
df_boats = pd.read_excel("data_cleaning_clickandboat_final_2026-07-24_26.xlsx")
df_boats = clean_columns(df_boats)

df_boats.to_sql(
    "clickandboat_barcos",
    engine,
    if_exists="replace",
    index=False
)

# 6. CARGAR Y SUBIR AEMET CLIMA (CSV)
df_aemet = pd.read_csv("aemet_clima_todos-a-abordo.csv")
df_aemet = clean_columns(df_aemet)

df_aemet.to_sql(
    "aemet_clima",
    engine,
    if_exists="replace",
    index=False
)

print("Tablas creadas/cargadas correctamente en MySQL ✅")

# 7. (OPCIONAL) PROBAR UNA QUERY RÁPIDA DESDE PYTHON
with engine.connect() as conn:
    result = conn.execute(text("SHOW TABLES;"))
    for row in result:
        print(row)


Tablas creadas/cargadas correctamente en MySQL ✅
('aemet_clima',)
('clickandboat_barcos',)
('turistas_internacionales_julio',)
('turistas_nacional_julio',)


In [ ]:
#escribe en la terminal de bash: python load_tables.ipynb    

In [ ]:
#luego ve a mysql workbench y escribe lo siguiente:

#CREATE DATABASE todos_a_bordo;
#USE todos_a_bordo;
#SHOW TABLES;

#y por último refresca myworkbench